## Simple benchmark using CatBoost

Benchamrk with CatBoost. item_cnt_month clipped between o to 20, validation on last month  of training data (33)
Final model is built using entire training data and final prediction is done on test data using that model.

In [1]:
import os
import sys

import pandas as pd
import xgboost as xgb

sys.path.insert(0, os.path.abspath('/home/jupyter/kaggle/predict_future_sales/src/'))

import common.com_util as util
import config.constants as constants
import cv.cv_util as cv
import munging.process_data as process_data
import modeling.train_util as model

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
logger = util.get_logger('sub_2')

train_df = pd.read_feather('/home/jupyter/kaggle/predict_future_sales/data/processed/train_all_merged.feather')
test_df = pd.read_feather('/home/jupyter/kaggle/predict_future_sales/data/processed/test_all_merged.feather')

train_features = ['shop_id', 'item_id', 'date_block_num', 'item_category_id', 'item_cnt_month']
train_df = train_df[train_features]

test_features = ['ID', 'shop_id', 'item_id', 'date_block_num', 'item_category_id']
test_df = test_df[test_features]

sample_submission = pd.read_feather('/home/jupyter/kaggle/predict_future_sales/data/processed/submission_processed.feather')

test = test_df.drop(['ID'], axis='columns')

# Clip the item_cnt_month within 20
train_df.item_cnt_month.clip(lower=0, upper=20, inplace=True)

/home/jupyter/anaconda3/envs/py37/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [4]:
TARGET = 'item_cnt_month'
ID = 'ID'
SEED = 42

training_months = [0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
holdout_months = [33]

training, validation = cv.get_data_splits_by_date_block(logger, train_df, 
                                                        train_months=training_months, 
                                                        validation_months=holdout_months)

[INFO]2020-09-10 09:45:37,534:sub_2:Splitting the data into train and holdout based on months...
[INFO]2020-09-10 09:45:37,535:sub_2:Training months [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
[INFO]2020-09-10 09:45:37,536:sub_2:Validation months [33]
[INFO]2020-09-10 09:45:38,322:sub_2:Shape of the training data (10675678, 5) 
[INFO]2020-09-10 09:45:38,323:sub_2:Shape of the validation data (238172, 5)


In [5]:
cat_params = {
                "objective": "RMSE", # Alias loss_function
                "custom_metric": "RMSE",
                "eval_metric": 'RMSE',
                "n_estimators": 10000,
                "learning_rate": 0.03,
                "random_seed": SEED,
                "early_stopping_rounds": 100,
                "use_best_model": True,
                "max_depth": 16,
                "max_bin": 254,
                "thread_count":-1,
                'boosting_type': 'Plain',
                'verbose_eval': 100,
                }

In [6]:
predictors = ['shop_id', 'item_id', 'date_block_num', 'item_category_id']

In [7]:
bst, best_iteration, valid_score, prediction = model.cat_train_validate_on_holdout(
        logger=logger, training=training, validation=validation, 
        predictors=predictors, target=TARGET, cat_features=predictors, params=cat_params, test_X=test
    )

[INFO]2020-09-10 09:45:38,384:sub_2:Training using CatBoost and validating on holdout
[INFO]2020-09-10 09:45:38,455:sub_2:Shape of train_X, train_Y, validation_X, validation_Y: (10675678, 4), (10675678,), (238172, 4), (238172,)
0:	learn: 1.2067754	test: 1.2050395	test1: 1.1338335	best: 1.1338335 (0)	total: 14.4s	remaining: 1d 16h 1m 45s
100:	learn: 0.8031744	test: 0.7508132	test1: 1.0307617	best: 1.0307617 (100)	total: 7m 30s	remaining: 12h 15m 37s
200:	learn: 0.7734562	test: 0.7249260	test1: 1.0203710	best: 1.0203710 (200)	total: 14m 30s	remaining: 11h 46m 54s
300:	learn: 0.7544277	test: 0.7128893	test1: 1.0163697	best: 1.0161926 (298)	total: 21m 34s	remaining: 11h 35m 3s
400:	learn: 0.7401116	test: 0.7055103	test1: 1.0143796	best: 1.0132060 (381)	total: 28m 31s	remaining: 11h 23m
Stopped by overfitting detector  (100 iterations wait)

bestTest = 1.013205988
bestIteration = 381

Shrink model to first 382 iterations.
[INFO]2020-09-10 10:20:33,975:sub_2:Validation Score 1.01320598836231

KeyboardInterrupt: 

In [14]:
bst, best_iteration, valid_score

(<xgboost.core.Booster at 0x7f103aefc090>, 158, 1.0116379)

In [15]:
prediction

array([0.38830838, 0.24504653, 0.13283464, ..., 0.09535095, 0.09556562,
       0.20091727], dtype=float32)

In [16]:
test_df.shape

(214200, 5)

In [17]:
sample_submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [18]:
submission = pd.DataFrame({'ID': test_df.ID, 'item_cnt_month': prediction})

In [19]:
submission.head()

,ID,item_cnt_month
0,0,0.388308
1,1,0.245047
2,2,0.132835
3,3,-0.011641
4,4,0.167597


In [8]:
cat_params

{'objective': 'RMSE',
 'custom_metric': 'RMSE',
 'eval_metric': 'RMSE',
 'n_estimators': 132,
 'learning_rate': 0.03,
 'random_seed': 42,
 'early_stopping_rounds': 100,
 'use_best_model': True,
 'max_depth': 16,
 'max_bin': 254,
 'thread_count': -1,
 'boosting_type': 'Plain',
 'verbose_eval': 100}